# Preprocessing Flux Distributions for Objective I #

In [1]:
import numpy as np
import cameo
import pandas as pd
from copy import deepcopy
import pickle
import time
import multiprocessing
from joblib import Parallel, delayed
from mcpecaso.plotting import multiplot_envelopes,plot_envelope,plot_pecaso_dfba,\
                              multi_two_stage_char_contours,two_stage_char_contour
from mcpecaso.core import mcPECASO
import mcpecaso as mcpecaso
from mcpecaso.core.substrate_dependent_envelopes import linear_uptake, logistic_uptake
import time


mcpecaso.settings.time_end = 100
mcpecaso.settings.initial_biomass = 0.05
mcpecaso.settings.initial_substrate = 500
mcpecaso.settings.num_timepoints = 10000
mcpecaso.settings.scope = 'extrema'

pd.options.display.max_rows = 100

num_cores = multiprocessing.cpu_count()

In [2]:
import sys
import colorlover as cl
from plotly import tools, subplots
import plotly.graph_objs as go
import pickle
import plotly.io as pio
pio.templates.default = "none"
import os

try:
    _ = __IPYTHON__
except NameError:
    from plotly.offline import plot
else:
    if 'ipykernel' in sys.modules:
        from plotly.offline import init_notebook_mode
        from plotly.offline import iplot as plot
        from IPython.display import HTML
        HTML("""
             <script>
              var waitForPlotly = setInterval( function() {
              if( typeof(window.Plotly) !== "undefined" ){
              MathJax.Hub.Config({ SVG: { font: "STIX-Web" }, displayAlign: "center" });
              MathJax.Hub.Queue(["setRenderer", MathJax.Hub, "SVG"]);
              clearInterval(waitForPlotly);}}, 250 );
            </script>
            """
        )
        init_notebook_mode(connected=True)
    elif 'IPython' in sys.modules:
        from plotly.offline import plot
    else:
        warn('Unknown ipython configuration')
        from plotly.offline import plot


In [3]:
pd.options.display.max_rows=600
pd.options.display.max_columns=150
wt_model = cameo.models.bigg.iJO1366

relevant_reactions = [reaction.id for reaction in wt_model.reactions 
                      if reaction.subsystem not in ['Transport, Outer Membrane',
                                                    'Transport, Inner Membrane',
                                                    'Inorganic Ion Transport and Metabolism',
                                                    'Transport, Outer Membrane Porin',
                                                    'Intracellular demand',
                                                    'Extracellular exchange']]
print(relevant_reactions)
relevant_reactions_set = set(relevant_reactions)

metabolite_names_dict = {'Spermidine': 'Spermidine',
                         'L-Tryptophan': 'L-Tryptophan',
                         'Thymidine C10H14N2O5': 'Thymidine',
                         'L-Phenylalanine': 'L-Phenylalanine',
                         'Adenosine': 'Adenosine',
                         'Indole': 'Indole',
                         'L-Tyrosine': 'L-Tyrosine',
                         'Inosine': 'Inosine',
                         'Xanthosine': 'Xanthosine',
                         'Cytidine': 'Cytidine',
                         'Uridine': 'Uridine',
                         'Quinate': 'Quinate',
                         'L-Isoleucine': 'L-Isoleucine',
                         'L-Lysine': 'L-Lysine',
                         'Hexanoate (n-C6:0)': 'Hexanoate',
                         'L-Leucine': 'L-Leucine',
                         'L-Arginine': 'L-Arginine',
                         'L-Histidine': 'L-Histidine',
                         'D-Gluconate': 'D-Gluconic Acid',
                         'L-Idonate': 'L-Idonic Acid',
                         '5-Dehydro-D-gluconate': '5-Ketogluconate',
                         'Citrate': 'Citric Acid',
                         'Agmatine': 'Agmatine',
                         'Ornithine': 'Ornithine',
                         'L-Proline': 'L-Proline',
                         'L-Valine': 'L-Valine',
                         'Thymine C5H6N2O2': 'Thymine',
                         'Adenine': 'Adenine',
                         'Guanine': 'Guanine',
                         'Hypoxanthine': 'Hypoxanthine',
                         'O-Acetyl-L-serine': 'O-Acetyl-L-serine',
                         'Xanthine': 'Xanthine',
                         'L-Glutamate': 'L-Glutamate',
                         '2-Oxoglutarate': '\u03b1-Ketoglutarate',
                         'Putrescine': 'Putrescine',
                         'L-Threonine': 'L-Threonine',
                         '4-Aminobutanoate': '\u03b3-Aminobutyrate',
                         'L-Homoserine': 'L-Homoserine',
                         'Allantoin': 'Allantoin',
                         'Succinate': 'Succinate',
                         'Uracil': 'Uracil',
                         'L-Asparagine': 'L-Asparagine',
                         'L-Malate': 'L-Malate',
                         'L-Aspartate': 'L-Aspartate',
                         'L-Cysteine': 'L-Cysteine',
                         'Glycerol 3-phosphate': 'Glycerol3-phosphate',
                         '3-Hydroxypropanoate': '3-Hydroxypropanoate',
                         '(S)-Propane-1,2-diol': '(S)-Propanediol',
                         '(R)-Propane-1,2-diol': '(R)-Propanediol',
                         'D-Alanine': 'D-Alanine',
                         'Glycerol': 'Glycerol',
                         '(R)-Glycerate': '(R)-Glycerate',
                         'D-Glyceraldehyde': 'Glyceraldehyde',
                         'L-Lactate': 'L-Lactate',
                         'Dihydroxyacetone': 'Dihydroxyacetone',
                         'L-Alanine': 'L-Alanine',
                         'D-Lactate': 'D-Lactate',
                         'L-Serine': 'L-Serine',
                         'Pyruvate': 'Pyruvate',
                         'Ethanolamine': 'Ethanolamine',
                         'Ethanol': 'Ethanol',
                         'Acetaldehyde': 'Acetaldehyde',
                         'Glycine': 'Glycine',
                         'Acetate': 'Acetate',
                         'Glycolate C2H3O3': 'Glycolate',
                         'Urea CH4N2O': 'Urea',
                         'Formate': 'Formate',
                         'Reduced glutathione': 'Glutathione',
                         '5-Methylthio-D-ribose': '5-Methylthioribose',
                         '1,5-Diaminopentane': 'Cadaverine'}

model = cameo.models.bigg.iJO1366
model.solver = 'gurobi'

metabolite_lookup_dict = {metabolite_names_dict[key]:key for key in metabolite_names_dict}

carbon_dict = {metabolite.name:str(metabolite.elements['C'])
               if 'C' in metabolite.elements else '0'
               for metabolite in model.metabolites}

carbon_dict = {metabolite:carbon_dict[metabolite] 
               if int(carbon_dict[metabolite])<=6 else '>6'
               for metabolite in metabolite_names_dict}


aa_metabolism_subsystems=['Alanine and Aspartate Metabolism',
                          'Arginine and Proline Metabolism',
                          'Cysteine Metabolism',
                          'Glutamate Metabolism',
                          'Glycine and Serine Metabolism',
                          'Histidine Metabolism',
                          'Methionine Metabolism',
                          'Threonine and Lysine Metabolism',
                          'Tyrosine, Tryptophan, and Phenylalanine Metabolism',
                          'Valine, Leucine, and Isoleucine Metabolism']
ijo_sub = {reaction.id:reaction.subsystem if reaction.subsystem not in aa_metabolism_subsystems
                                                    else 'Amino Acid Metabolism'
           for reaction in model.reactions}


['BIOMASS_Ec_iJO1366_WT_53p95M', 'BIOMASS_Ec_iJO1366_core_53p95M', '23PDE2pp', '23PDE4pp', '23PDE7pp', '23PDE9pp', '2AGPEAT120', '2AGPEAT140', '2AGPEAT141', '2AGPEAT160', '2AGPEAT161', '2AGPEAT180', '2AGPEAT181', '2AGPGAT120', '2AGPGAT140', '2AGPGAT141', '2AGPGAT160', '2AGPGAT161', '2AGPGAT180', '2AGPGAT181', '2DGULRGx', '2DGULRGy', '2DGULRx', '2DGULRy', '2MAHMP', '3AMACHYD', '3HAD100', '3HAD120', '5DOAN', '3HAD121', '3HAD140', 'A5PISO', 'AACPS1', 'AACPS2', '3HAD141', 'AACPS3', 'AACPS4', 'AACPS5', '3HAD160', 'AACPS6', 'AACPS7', '3HAD161', 'AACPS8', 'AACPS9', '3HAD180', '3HAD181', 'AACTOOR', 'AADDGT', 'AAMYL', 'AAMYLpp', 'AB6PGH', 'ABTA', 'ABUTD', '3HAD40', '3HAD60', '3HAD80', '3HCINNMH', '3HPPPNH', 'ACACCT', 'ACACT1r', '3KGK', 'ACACT2r', 'ACACT3r', 'ACACT4r', '3NTD2pp', '3NTD4pp', 'ACACT5r', 'ACACT6r', '3NTD7pp', '3NTD9pp', '3OAR100', 'ACACT7r', 'ACACT8r', '3OAR120', '3OAR121', '3OAR140', 'ACALD', '3OAR141', '3OAR160', '3OAR161', '3OAR180', 'ACANTHAT', '3OAR181', 'ACBIPGT', 'ACCOAC', '

SolverNotFound: gurobi is not a valid solver interface. Pick from ['glpk_exact', 'glpk', 'scipy'].

In [5]:
pickling_off = open("flux_dicts.pickle","rb")
flux_dicts = pickle.load(pickling_off)
pickling_off.close()

In [6]:
flux_dict_prod = flux_dicts['productivity']
flux_dict_prod_75_yield = flux_dicts['productivity_75_yield']
flux_dict_yield_2_prod = flux_dicts['yield_2_productivity']

In [4]:
def reduced_fva(flux_dict, product, used_reactions, unused_reactions, template_model, slack=0):
    
    model = deepcopy(template_model)
    
    biomass_flux, substrate_flux, target_flux = flux_dict[product]
    target_rxn = [reaction for reaction in model.exchanges 
                  if reaction.reactants[0].name==metabolite_lookup_dict[product]][0]
    
    with model:
        for reaction in list(unused_reactions):
                if model.reactions.get_by_id(reaction).reversibility:
                    model.reactions.get_by_id(reaction).bounds = (-slack, slack)
                else:
                    model.reactions.get_by_id(reaction).bounds = (0, slack)

         #CONSTRAINTS
        model.reactions.EX_glc__D_e.bounds = (substrate_flux-slack, 1000)
        model.reactions.get_by_id(target_rxn.id).bounds = (target_flux-slack, target_flux+slack)

        mut_df = cameo.flux_variability_analysis(model,fraction_of_optimum=1,reactions=used_reactions).data_frame
        mut_pfba = cameo.pfba(model).data_frame.reindex(used_reactions)

    mut_pfba[mut_pfba.abs()<0.001]=0
    mut_df[mut_df.abs()<0.001]=0
    temp = pd.concat([mut_pfba,mut_df], axis=1, sort=True)
    temp.columns = ['pfba','lb','ub']
    temp['range'] = temp.ub-temp.lb
    temp[temp.abs()<0.001]=0
    
    return {product:temp}

In [5]:
def global_fva_parser(fva_dict, flex_cutoff=0.001):
    parsed_fva = {}
    temp_dict = deepcopy(fva_dict)
    for product in temp_dict.keys():
        temp_dict[product] = temp_dict[product].fillna(0)
        pfba_values = temp_dict[product][(temp_dict[product].range<=flex_cutoff)].pfba.values
        temp_dict[product].loc[temp_dict[product][(temp_dict[product].range<=flex_cutoff)].index,'lb'] = pfba_values
        temp_dict[product].loc[temp_dict[product][(temp_dict[product].range<=flex_cutoff)].index,'ub'] = pfba_values
        temp_dict[product].loc[temp_dict[product][(temp_dict[product].range<=flex_cutoff)].index,'range'] = 0
    
    
    products = list(temp_dict.keys())

    range_df = pd.concat([temp_dict[product].range for product in temp_dict.keys()],axis=1)
    range_df = range_df.fillna(0)
    flexible_reactions_set = set(range_df[range_df.any(axis=1)].index)
    inflexible_reactions_set = relevant_reactions_set - flexible_reactions_set
    
    
    pfba_df = pd.concat([temp_dict[product].pfba for product in temp_dict.keys()],axis=1)
    pfba_df = pfba_df.fillna(0)
    pfba_df.columns = products
    
    used_reactions_set = set(pfba_df[pfba_df.any(axis=1)].index)
    unused_reactions_set = relevant_reactions_set - used_reactions_set

    parsed_fva['used_reactions_set'] = used_reactions_set
    parsed_fva['unused_reactions_set'] = unused_reactions_set
    parsed_fva['flexible_reactions_set'] = flexible_reactions_set
    parsed_fva['inflexible_reactions_set'] = inflexible_reactions_set
    parsed_fva['inflexible'] = {}
    parsed_fva['inflexible']['pfba'] = pd.concat([temp_dict[product].reindex(inflexible_reactions_set.intersection(used_reactions_set)).pfba for product in temp_dict.keys()],axis=1)
    parsed_fva['inflexible']['pfba'].columns = products
    parsed_fva['flexible'] = {}
    parsed_fva['flexible']['pfba'] = pd.concat([temp_dict[product].reindex(flexible_reactions_set.intersection(used_reactions_set)).pfba for product in temp_dict.keys()],axis=1)
    parsed_fva['flexible']['pfba'].columns = products
    columns = []
    for product in temp_dict.keys():
        columns.append((product,'lb'))
        columns.append((product,'ub'))
    columns = pd.MultiIndex.from_tuples(columns)
    parsed_fva['flexible']['bounds'] = pd.concat([temp_dict[product].reindex(flexible_reactions_set.intersection(used_reactions_set))[['lb','ub']] for product in temp_dict.keys()], axis=1)
    parsed_fva['flexible']['bounds'].columns = columns
    parsed_fva['flexible']['range'] = pd.concat([temp_dict[product].reindex(flexible_reactions_set.intersection(used_reactions_set)).range for product in temp_dict.keys()],axis=1)
    parsed_fva['flexible']['range'].columns = products
    
    #products flexible, products inflexible but active, products inflexible AND inactive
    flex_products = []
    flex_products_no = []
    inflex_used_products = []
    inflex_used_products_no = []
    inflex_unused_products = []
    inflex_unused_products_no = []
    for reaction in parsed_fva['flexible']['pfba'].index:
        flex_products.append(set(parsed_fva['flexible']['range'].loc[reaction][parsed_fva['flexible']['range'].loc[reaction]!=0].index))
        flex_products_no.append(len(parsed_fva['flexible']['range'].loc[reaction][parsed_fva['flexible']['range'].loc[reaction]!=0].index))
        inflex_products = parsed_fva['flexible']['range'].loc[reaction][parsed_fva['flexible']['range'].loc[reaction]==0].index
        inflex_used_products.append(set(parsed_fva['flexible']['pfba'].loc[reaction].reindex(inflex_products)[parsed_fva['flexible']['pfba'].loc[reaction].reindex(inflex_products)!=0].index))
        inflex_used_products_no.append(len(parsed_fva['flexible']['pfba'].loc[reaction].reindex(inflex_products)[parsed_fva['flexible']['pfba'].loc[reaction].reindex(inflex_products)!=0].index))
        inflex_unused_products.append(set(parsed_fva['flexible']['pfba'].loc[reaction].reindex(inflex_products)[parsed_fva['flexible']['pfba'].loc[reaction].reindex(inflex_products)==0].index))
        inflex_unused_products_no.append(len(parsed_fva['flexible']['pfba'].loc[reaction].reindex(inflex_products)[parsed_fva['flexible']['pfba'].loc[reaction].reindex(inflex_products)==0].index))

    parsed_fva['flexible']['products'] = pd.DataFrame(data={'flex':flex_products, 'flex_no':flex_products_no, 'inflex_used':inflex_used_products,
                                                 'inflex_used_no':inflex_used_products_no, 'inflex_unused':inflex_unused_products,
                                                 'inflex_unused_no':inflex_unused_products_no}, index=parsed_fva['flexible']['pfba'].index)

    
    sorted_order = parsed_fva['flexible']['products'].sort_values(by=['flex_no','inflex_used_no'],ascending=[False,False]).index
    parsed_fva['flexible']['pfba'] = parsed_fva['flexible']['pfba'].reindex(sorted_order)
    parsed_fva['flexible']['bounds'] = parsed_fva['flexible']['bounds'].reindex(sorted_order) 
    parsed_fva['flexible']['range'] = parsed_fva['flexible']['range'].reindex(sorted_order)
    parsed_fva['flexible']['products'] = parsed_fva['flexible']['products'].reindex(sorted_order) 
    
    
    return temp_dict,parsed_fva

# FIRST PASS FVA #

In [32]:
#FIRST PASS FVA - NO EXTRA CONSTRAINTS APART FROM WILDTYPE. FVA DONE ONLY ON RELEVANT REACTIO
model.solver='cplex'

start_time = time.time()
result_list = Parallel(n_jobs=num_cores, verbose=20)(delayed(reduced_fva)(flux_dict_prod, product, relevant_reactions_set,
                                                                         [], model, slack=0) for product in flux_dict_prod.keys())


fva = {list(result_dict.keys())[0]: list(result_dict.values())[0] 
                                for result_dict in result_list}

print(time.time()-start_time)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:   46.1s
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   50.5s
[Parallel(n_jobs=8)]: Done   3 tasks      | elapsed:   52.0s
[Parallel(n_jobs=8)]: Done   4 tasks      | elapsed:   56.0s
[Parallel(n_jobs=8)]: Done   5 tasks      | elapsed:   58.0s
[Parallel(n_jobs=8)]: Done   6 tasks      | elapsed:  1.0min
[Parallel(n_jobs=8)]: Done   7 tasks      | elapsed:  1.1min
[Parallel(n_jobs=8)]: Done   8 tasks      | elapsed:  1.1min
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:  1.8min
[Parallel(n_jobs=8)]: Done  10 tasks      | elapsed:  1.8min
[Parallel(n_jobs=8)]: Done  11 tasks      | elapsed:  1.9min
[Parallel(n_jobs=8)]: Done  12 tasks      | elapsed:  1.9min
[Parallel(n_jobs=8)]: Done  13 tasks      | elapsed:  2.0min
[Parallel(n_jobs=8)]: Done  14 tasks      | elapsed:  2.0min
[Parallel(n_jobs=8)]: Done  15 tasks      | elapsed:  2.1min
[Parallel(

KeyboardInterrupt: 

In [18]:
pickling_on = open("fva.pickle","wb")
#pickle.dump(fva, pickling_on)
pickling_on.close() 

In [33]:
pickle_off = open("fva.pickle","rb")
fva = pickle.load(pickle_off)
pickle_off.close()

In [34]:
reactions_set = set([reaction.id for reaction in model.reactions])
relevant_reactions_set = set(relevant_reactions)
irrelevant_reactions_set = reactions_set-relevant_reactions_set

fva,parsed_fva = global_fva_parser(fva,flex_cutoff=0.001)

print("Total Reactions: ",len(reactions_set), "\n")
print("Irrelevant_reactions: ",len(irrelevant_reactions_set))
print("Relevant Reactions: ",len(relevant_reactions_set), "\n")
print("Inflexible Reactions: ",len(parsed_fva['inflexible_reactions_set']))
print("Unused Inflexible Reactions: ",len(parsed_fva['inflexible_reactions_set'].intersection(parsed_fva['unused_reactions_set'])))
print("Used Inflexible Reactions: ",len(parsed_fva['inflexible_reactions_set'].intersection(parsed_fva['used_reactions_set'])), "\n")
print("Flexible Reactions: ",len(parsed_fva['flexible_reactions_set']))
print("Unused Flexible Reactions: ",len(parsed_fva['flexible_reactions_set'].intersection(parsed_fva['unused_reactions_set'])))
print("Used Flexible Reactions: ",len(parsed_fva['flexible_reactions_set'].intersection(parsed_fva['used_reactions_set'])))

unused_reactions_set_1 = parsed_fva['unused_reactions_set']
used_reactions_set_1 = parsed_fva['used_reactions_set']

Total Reactions:  2583 

Irrelevant_reactions:  1090
Relevant Reactions:  1493 

Inflexible Reactions:  1202
Unused Inflexible Reactions:  970
Used Inflexible Reactions:  232 

Flexible Reactions:  291
Unused Flexible Reactions:  138
Used Flexible Reactions:  153


In [57]:
wt_model.solver = 'gurobi'
wt_df = cameo.flux_variability_analysis(wt_model,fraction_of_optimum=1,reactions=relevant_reactions).data_frame
wt_pfba = cameo.pfba(wt_model).data_frame.reindex(relevant_reactions)

#Pruning out small values. If a flux is less than 0.001, make it equal to 0
wt_pfba[wt_pfba.abs()<0.001]=0
wt_df[wt_df.abs()<0.001]=0

#Consolidate pfba and fva into a single dataframe
comparison_df = pd.concat([wt_pfba,wt_df], axis=1, sort=True)
comparison_df.columns = ['pfba','lb','ub']

#Add a column describing the range of allowable fluxes for each reaction
comparison_df['range'] = comparison_df.ub-comparison_df.lb
wt_fva = comparison_df
wt_used_reactions = set(list(wt_fva[wt_fva.pfba!=0].index))

# SECOND PASS FVA #

In [31]:
#SECOND PASS FVA - REACTIONS UNUSED FOR ALL PRODUCTS FROM FIRST PASS CONSTRAINED TO ZERO
start_time = time.time()

result_list = Parallel(n_jobs=num_cores, verbose=20)(delayed(reduced_fva)(flux_dict_prod, product, used_reactions=used_reactions_set_1,
                                                                         unused_reactions=unused_reactions_set_1,
                                                                          template_model=model, slack=0.001) for product in flux_dict_prod.keys())


fva_second = {list(result_dict.keys())[0]: list(result_dict.values())[0] 
                                for result_dict in result_list}

print("Completed in ", time.time()-start_time, " s")

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:   36.0s
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   40.2s
[Parallel(n_jobs=8)]: Done   3 tasks      | elapsed:   41.8s
[Parallel(n_jobs=8)]: Done   4 tasks      | elapsed:   47.6s
[Parallel(n_jobs=8)]: Done   5 tasks      | elapsed:   48.8s
[Parallel(n_jobs=8)]: Done   6 tasks      | elapsed:   53.7s
[Parallel(n_jobs=8)]: Done   7 tasks      | elapsed:  1.0min
[Parallel(n_jobs=8)]: Done   8 tasks      | elapsed:  1.1min
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:  1.2min
[Parallel(n_jobs=8)]: Done  10 tasks      | elapsed:  1.3min
[Parallel(n_jobs=8)]: Done  11 tasks      | elapsed:  1.4min
[Parallel(n_jobs=8)]: Done  12 tasks      | elapsed:  1.4min
[Parallel(n_jobs=8)]: Done  13 tasks      | elapsed:  1.5min
[Parallel(n_jobs=8)]: Done  14 tasks      | elapsed:  1.6min
[Parallel(n_jobs=8)]: Done  15 tasks      | elapsed:  1.6min
[Parallel(

Completed in  348.7808129787445  s


[Parallel(n_jobs=8)]: Done  70 out of  70 | elapsed:  5.8min finished


In [35]:
pickling_on = open("fva_second.pickle","wb")
#pickle.dump(fva_second, pickling_on)
pickling_on.close() 

In [6]:
pickle_off = open("fva_second.pickle","rb")
fva_second = pickle.load(pickle_off)
pickle_off.close()

In [7]:
reactions_set = set([reaction.id for reaction in model.reactions])
relevant_reactions_set = set(relevant_reactions)
irrelevant_reactions_set = reactions_set-relevant_reactions_set

fva_second,parsed_fva_2 = global_fva_parser(fva_second,flex_cutoff=0.008)

print("Total Reactions: ",len(reactions_set), "\n")
print("Irrelevant_reactions: ",len(irrelevant_reactions_set))
print("Relevant Reactions: ",len(relevant_reactions_set), "\n")
print("Inflexible Reactions: ",len(parsed_fva_2['inflexible_reactions_set']))
print("Unused Inflexible Reactions: ",len(parsed_fva_2['inflexible_reactions_set'].intersection(parsed_fva_2['unused_reactions_set'])))
print("Used Inflexible Reactions: ",len(parsed_fva_2['inflexible_reactions_set'].intersection(parsed_fva_2['used_reactions_set'])), "\n")
print("Flexible Reactions: ",len(parsed_fva_2['flexible_reactions_set']))
print("Unused Flexible Reactions: ",len(parsed_fva_2['flexible_reactions_set'].intersection(parsed_fva_2['unused_reactions_set'])))
print("Used Flexible Reactions: ",len(parsed_fva_2['flexible_reactions_set'].intersection(parsed_fva_2['used_reactions_set'])))

unused_reactions_set_2 = parsed_fva_2['unused_reactions_set']
used_reactions_set_2 = parsed_fva_2['used_reactions_set']

Total Reactions:  2583 

Irrelevant_reactions:  1090
Relevant Reactions:  1493 

Inflexible Reactions:  1399
Unused Inflexible Reactions:  1108
Used Inflexible Reactions:  291 

Flexible Reactions:  94
Unused Flexible Reactions:  0
Used Flexible Reactions:  94


In [38]:
parsed_fva_2['flexible']['products']

,flex,flex_no,inflex_used,inflex_used_no,inflex_unused,inflex_unused_no
DHORD2,"{Ethanolamine, L-Isoleucine, Dihydroxyacetone,...",70,{},0,{},0
SUCDi,"{Ethanolamine, L-Isoleucine, Dihydroxyacetone,...",70,{},0,{},0
NADH17pp,"{Ethanolamine, L-Isoleucine, Dihydroxyacetone,...",70,{},0,{},0
R1PK,"{Ethanolamine, L-Isoleucine, Dihydroxyacetone,...",70,{},0,{},0
FRD2,"{Ethanolamine, L-Isoleucine, Dihydroxyacetone,...",70,{},0,{},0
FBA,"{Ethanolamine, L-Isoleucine, Dihydroxyacetone,...",70,{},0,{},0
R15BPK,"{Ethanolamine, L-Isoleucine, Dihydroxyacetone,...",70,{},0,{},0
TALA,"{Ethanolamine, L-Isoleucine, Dihydroxyacetone,...",70,{},0,{},0
ADK1,"{Ethanolamine, L-Isoleucine, Dihydroxyacetone,...",70,{},0,{},0
NADH16pp,"{Ethanolamine, L-Isoleucine, Dihydroxyacetone,...",70,{},0,{},0


# THIRD PASS FVA #

In [40]:
#THIRD PASS FVA - REACTIONS UNUSED FOR ALL PRODUCTS FROM SECOND PASS CONSTRAINED TO ZERO
start_time = time.time()

result_list = Parallel(n_jobs=num_cores, verbose=20)(delayed(reduced_fva)(flux_dict_prod, product, used_reactions=used_reactions_set_2,
                                                                         unused_reactions=unused_reactions_set_2,
                                                                          template_model=model, slack=0.001) for product in flux_dict_prod.keys())


fva_third = {list(result_dict.keys())[0]: list(result_dict.values())[0] 
                                for result_dict in result_list}

print("Completed in ", time.time()-start_time, " s")

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:   39.3s
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   41.6s
[Parallel(n_jobs=8)]: Done   3 tasks      | elapsed:   45.5s
[Parallel(n_jobs=8)]: Done   4 tasks      | elapsed:   47.8s
[Parallel(n_jobs=8)]: Done   5 tasks      | elapsed:   51.4s
[Parallel(n_jobs=8)]: Done   6 tasks      | elapsed:   54.3s
[Parallel(n_jobs=8)]: Done   7 tasks      | elapsed:  1.0min
[Parallel(n_jobs=8)]: Done   8 tasks      | elapsed:  1.1min
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:  1.2min
[Parallel(n_jobs=8)]: Done  10 tasks      | elapsed:  1.3min
[Parallel(n_jobs=8)]: Done  11 tasks      | elapsed:  1.4min
[Parallel(n_jobs=8)]: Done  12 tasks      | elapsed:  1.4min
[Parallel(n_jobs=8)]: Done  13 tasks      | elapsed:  1.4min
[Parallel(n_jobs=8)]: Done  14 tasks      | elapsed:  1.5min
[Parallel(n_jobs=8)]: Done  15 tasks      | elapsed:  1.6min
[Parallel(

Completed in  355.50414061546326  s


[Parallel(n_jobs=8)]: Done  70 out of  70 | elapsed:  5.9min finished


In [41]:
pickling_on = open("fva_third.pickle","wb")
#pickle.dump(fva_third, pickling_on)
pickling_on.close()

In [42]:
pickle_off = open("fva_third.pickle","rb")
fva_third = pickle.load(pickle_off)
pickle_off.close()

In [43]:
reactions_set = set([reaction.id for reaction in model.reactions])
relevant_reactions_set = set(relevant_reactions)
irrelevant_reactions_set = reactions_set-relevant_reactions_set

fva_third_temp,parsed_fva_3 = global_fva_parser(fva_third,flex_cutoff=0.008)

print("Total Reactions: ",len(reactions_set), "\n")
print("Irrelevant_reactions: ",len(irrelevant_reactions_set))
print("Relevant Reactions: ",len(relevant_reactions_set), "\n")
print("Inflexible Reactions: ",len(parsed_fva_3['inflexible_reactions_set']))
print("Unused Inflexible Reactions: ",len(parsed_fva_3['inflexible_reactions_set'].intersection(parsed_fva_3['unused_reactions_set'])))
print("Used Inflexible Reactions: ",len(parsed_fva_3['inflexible_reactions_set'].intersection(parsed_fva_3['used_reactions_set'])), "\n")
print("Flexible Reactions: ",len(parsed_fva_3['flexible_reactions_set']))
print("Unused Flexible Reactions: ",len(parsed_fva_3['flexible_reactions_set'].intersection(parsed_fva_3['unused_reactions_set'])))
print("Used Flexible Reactions: ",len(parsed_fva_3['flexible_reactions_set'].intersection(parsed_fva_3['used_reactions_set'])))

unused_reactions_set_3 = parsed_fva_3['unused_reactions_set']
used_reactions_set_3 = parsed_fva_3['used_reactions_set']

Total Reactions:  2583 

Irrelevant_reactions:  1090
Relevant Reactions:  1493 

Inflexible Reactions:  1399
Unused Inflexible Reactions:  1108
Used Inflexible Reactions:  291 

Flexible Reactions:  94
Unused Flexible Reactions:  0
Used Flexible Reactions:  94


In [44]:
parsed_fva_3['flexible']['products']

,flex,flex_no,inflex_used,inflex_used_no,inflex_unused,inflex_unused_no
DHORD2,"{Ethanolamine, L-Isoleucine, Dihydroxyacetone,...",70,{},0,{},0
SUCDi,"{Ethanolamine, L-Isoleucine, Dihydroxyacetone,...",70,{},0,{},0
NADH17pp,"{Ethanolamine, L-Isoleucine, Dihydroxyacetone,...",70,{},0,{},0
R1PK,"{Ethanolamine, L-Isoleucine, Dihydroxyacetone,...",70,{},0,{},0
FRD2,"{Ethanolamine, L-Isoleucine, Dihydroxyacetone,...",70,{},0,{},0
FBA,"{Ethanolamine, L-Isoleucine, Dihydroxyacetone,...",70,{},0,{},0
R15BPK,"{Ethanolamine, L-Isoleucine, Dihydroxyacetone,...",70,{},0,{},0
TALA,"{Ethanolamine, L-Isoleucine, Dihydroxyacetone,...",70,{},0,{},0
ADK1,"{Ethanolamine, L-Isoleucine, Dihydroxyacetone,...",70,{},0,{},0
NADH16pp,"{Ethanolamine, L-Isoleucine, Dihydroxyacetone,...",70,{},0,{},0


# FOURTH PASS FVA #

In [92]:
parsed_fva_3_temp = deepcopy(parsed_fva_3)
fva_third_temp = deepcopy(fva_third)

In [93]:
parsed_fva_3['flexible']['products'][parsed_fva_3_temp['flexible']['products'].inflex_used_no==0].index

Index(['DHORD2', 'SUCDi', 'NADH17pp', 'R1PK', 'FRD2', 'FBA', 'R15BPK', 'TALA',
       'ADK1', 'NADH16pp', 'PRPPS', 'PPM', 'DHORDfum', 'FBA3', 'PFK', 'PFK_3',
       'HEX1', 'XYLI2', 'HEX7', 'GART', 'FTHFLi', 'THRD', 'THRA', 'RNDR3',
       'RNDR2', 'NDPK6', 'NDPK5', 'FLDR2', 'RNDR4', 'NDPK7', 'FADRx2', 'AMPN',
       'NTD7', 'TRPS1', 'IDOND', 'IDOND2', 'SERD_L', 'FBP'],
      dtype='object')

In [60]:
#FOURTH PASS FVA - FLEXIBLE REACTIONS THAT CAN BE ZERO FOR ALL PRODUCTS REMOVED ONE BY ONE AND FVA UPDATED AFTER EACH REMOVAL

deletable_reactions = parsed_fva_3['flexible']['products'][parsed_fva_3_temp['flexible']['products'].inflex_used_no==0].index

for reaction in reversed(deletable_reactions):
    if (reaction in parsed_fva_3_temp['flexible']['products'].index) and (reaction not in wt_used_reactions):
        zeroflag=1
        flex_products = [metabolite_lookup_dict[product] for product in parsed_fva_3_temp['flexible']['products']['flex'].loc[reaction]]
        for product in flex_products:
            if not((parsed_fva_3_temp['flexible']['bounds'].loc[reaction].loc[metabolite_names_dict[product]].lb<=0) and (parsed_fva_3_temp['flexible']['bounds'].loc[reaction].loc[metabolite_names_dict[product]].ub>=0)):
                zeroflag=0
        if zeroflag:
            print("Removing reaction: ", reaction)
            used_reactions_set = parsed_fva_3_temp['used_reactions_set'] - set([reaction])
            unused_reactions_set = parsed_fva_3_temp['unused_reactions_set'].union(set([reaction]))
            for i,product in enumerate(flex_products):
                print('Evaluating Product ', i+1, ' of ', len(flex_products))
                temp = reduced_fva(flux_dict_prod, metabolite_names_dict[product],
                                                                             unused_reactions=unused_reactions_set,
                                                                             used_reactions=used_reactions_set,
                                                                             template_model=model, slack=0.001)
                fva_third_temp[metabolite_names_dict[product]] = list(temp.values())[0]
            
            fva_third_temp,parsed_fva_3_temp = global_fva_parser(fva_third_temp,flex_cutoff=0.008)


Removing reaction:  IDOND2
Evaluating Product  1  of  2
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpe9pmv0er.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  2  of  2
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp16ypnf4w.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros


C:\Users\kraj5\Anaconda3\envs\dyssco_plotly_4\lib\site-packages\ipykernel_launcher.py:14: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.


C:\Users\kraj5\Anaconda3\envs\dyssco_plotly_4\lib\site-packages\ipykernel_launcher.py:20: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




Removing reaction:  TRPS1
Evaluating Product  1  of  2
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp4raeec_v.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  2  of  2
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpgyzgsz4u.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  NTD7
Evaluating Product  1  of  2
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpdljuc2bk.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  2  of  2
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpbkho22fy.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  FADRx2
Evaluating Product  1  of  2
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp6ipz1zhj.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  2  of  

Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  HEX7
Evaluating Product  1  of  55
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpuahimk3v.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  2  of  55
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpofcrr3xc.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  3  of  55
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp7tqnjiql.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  4  of  55
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmptgx69cam.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  5  of  55
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpf6dwu5_h.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evalu

Evaluating Product  46  of  55
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpbamrtk1r.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  47  of  55
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp6ybf872g.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  48  of  55
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmphmwdjm_w.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  49  of  55
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmps_j2jizf.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  50  of  55
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmppaal_6ox.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  51  of  55
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp

Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpyk0lyrqs.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  37  of  70
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmplpvwpczh.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  38  of  70
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp4sd13nte.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  39  of  70
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpjkyisogc.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  40  of  70
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp59zrlbxm.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  41  of  70
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpg7ve4eh4.lp
Reading time = 

Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  12  of  70
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpha4q0add.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  13  of  70
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpaquyx3if.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  14  of  70
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp2h3e28n_.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  15  of  70
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpnuwmy7g4.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  16  of  70
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp1es2iikp.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  17  o

Evaluating Product  57  of  70
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp8asf7u8x.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  58  of  70
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpgbmg50h_.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  59  of  70
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp__j0bq02.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  60  of  70
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp4txx_stn.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  61  of  70
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp90n7of9w.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  62  of  70
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp

In [61]:
fva_fourth = deepcopy(fva_third_temp)


In [62]:
pickling_on = open("fva_fourth.pickle","wb")
#pickle.dump(fva_fourth, pickling_on)
pickling_on.close() 

In [63]:
pickle_off = open("fva_fourth.pickle","rb")
fva_fourth = pickle.load(pickle_off)
pickle_off.close()

In [64]:
reactions_set = set([reaction.id for reaction in model.reactions])
relevant_reactions_set = set(relevant_reactions)
irrelevant_reactions_set = reactions_set-relevant_reactions_set

fva_fourth,parsed_fva_4 = global_fva_parser(fva_fourth,flex_cutoff=0.008)

print("Total Reactions: ",len(reactions_set), "\n")
print("Irrelevant_reactions: ",len(irrelevant_reactions_set))
print("Relevant Reactions: ",len(relevant_reactions_set), "\n")
print("Inflexible Reactions: ",len(parsed_fva_4['inflexible_reactions_set']))
print("Unused Inflexible Reactions: ",len(parsed_fva_4['inflexible_reactions_set'].intersection(parsed_fva_4['unused_reactions_set'])))
print("Used Inflexible Reactions: ",len(parsed_fva_4['inflexible_reactions_set'].intersection(parsed_fva_4['used_reactions_set'])), "\n")
print("Flexible Reactions: ",len(parsed_fva_4['flexible_reactions_set']))
print("Unused Flexible Reactions: ",len(parsed_fva_4['flexible_reactions_set'].intersection(parsed_fva_4['unused_reactions_set'])))
print("Used Flexible Reactions: ",len(parsed_fva_4['flexible_reactions_set'].intersection(parsed_fva_4['used_reactions_set'])))

unused_reactions_set_4 = parsed_fva_4['unused_reactions_set']
used_reactions_set_4 = parsed_fva_4['used_reactions_set']

Total Reactions:  2583 

Irrelevant_reactions:  1090
Relevant Reactions:  1493 

Inflexible Reactions:  1434
Unused Inflexible Reactions:  1120
Used Inflexible Reactions:  314 

Flexible Reactions:  59
Unused Flexible Reactions:  0
Used Flexible Reactions:  59


In [65]:
parsed_fva_4['flexible']['products']

,flex,flex_no,inflex_used,inflex_used_no,inflex_unused,inflex_unused_no
SUCDi,"{Ethanolamine, L-Isoleucine, Dihydroxyacetone,...",70,{},0,{},0
NADH17pp,"{Ethanolamine, L-Isoleucine, Dihydroxyacetone,...",70,{},0,{},0
TALA,"{Ethanolamine, L-Isoleucine, Dihydroxyacetone,...",70,{},0,{},0
NADH16pp,"{Ethanolamine, L-Isoleucine, Dihydroxyacetone,...",70,{},0,{},0
FRD2,"{Ethanolamine, L-Isoleucine, Dihydroxyacetone,...",70,{},0,{},0
FBA,"{Ethanolamine, L-Isoleucine, Dihydroxyacetone,...",70,{},0,{},0
PFK,"{Ethanolamine, L-Isoleucine, Spermidine, 3-Hyd...",69,{},0,{Dihydroxyacetone},1
PFK_3,"{Ethanolamine, L-Isoleucine, Spermidine, 3-Hyd...",69,{},0,{Dihydroxyacetone},1
FBA3,"{Ethanolamine, L-Isoleucine, Spermidine, 3-Hyd...",69,{},0,{Dihydroxyacetone},1
MDH,"{L-Idonic Acid, 5-Ketogluconate, Dihydroxyacet...",3,"{Ethanolamine, L-Isoleucine, Spermidine, 3-Hyd...",67,{},0


# Fifth Pass FVA - Product Specific Deletions#

In [66]:
# Identifies product specific constraints that are possible:
# From fourth pass FVA, perform following steps:
# 1. Identify which products still have flexible reactions and iterate through each:
# 2. For each flexible reaction in that product:
# 3. Count products whose PFBA result falls within the flux range for that reaction
# 4. Also, identify the most frequently occurring flux value and the number of times it appears.
# 5. Arrange in descending order of number of times the most frequently occurring flux value occurs.

complete_products_set = set(fva_fourth.keys())
flex_products_lists = list(parsed_fva_4['flexible']['products']['flex'].values)
flex_products_sets = set([frozenset(product_list) for product_list in flex_products_lists])
temp_list = []
for temp_set in flex_products_sets:
    temp_list.extend(list(temp_set))
flex_products = (set(temp_list))
temp_dict = {}
for i,product in enumerate(flex_products):
    print('Analyzing Product :', product)
    reactions = parsed_fva_4['flexible']['range'][product][parsed_fva_4['flexible']['range'][product]!=0].index
    products_within_bounds = []
    products_within_bounds_no = []
    mode_pfba = []
    mode_no =[]
    for reaction in reactions:
        ub = parsed_fva_4['flexible']['bounds'].loc[reaction][product]['ub']
        lb = parsed_fva_4['flexible']['bounds'].loc[reaction][product]['lb']
        pfba = parsed_fva_4['flexible']['pfba'].loc[reaction][complete_products_set-set([product])]
        temp_df = pfba[(pfba>=lb)&(pfba<=ub)]
        temp_df = temp_df.round(decimals=3)
        products_within_bounds.append(list(temp_df.index))
        products_within_bounds_no.append(len(temp_df.index))
        if len(temp_df.index):
            mode_pfba.append(temp_df.mode().values[0])
            mode_no.append(len(temp_df[temp_df==temp_df.mode().values[0]].index))
        else:
            mode_pfba.append(np.nan)
            mode_no.append(0)
    data = {'products_within_bounds':products_within_bounds,'number':products_within_bounds_no,'mode_pfba':mode_pfba,'mode_no':mode_no}
    df = pd.DataFrame(data,index=reactions)
    temp_df = pd.concat([parsed_fva_4['flexible']['bounds'].loc[reactions][product],parsed_fva_4['flexible']['pfba'].loc[reactions][product]],axis=1)
    product_df = pd.concat([temp_df,df],axis=1)
    product_df = product_df.sort_values(by='mode_no',ascending=False)
    product_df.columns = ['lower','upper','pfba','products_within_bounds','number_within_bounds','mode','no_mode']
    temp_dict[product]=product_df
    display(product_df)

Analyzing Product : Ethanolamine


,lower,upper,pfba,products_within_bounds,number_within_bounds,mode,no_mode
FRD2,0.000000,26.972834,0.000000,"[L-Isoleucine, Hexanoate, Ornithine, D-Gluconi...",69,0.000,68
NADH17pp,0.000000,26.971191,0.001000,"[L-Isoleucine, Hexanoate, Ornithine, D-Gluconi...",69,0.000,59
TALA,-4.697628,0.996770,0.000000,"[L-Isoleucine, Hexanoate, Ornithine, D-Gluconi...",68,0.000,48
PFK_3,0.000000,5.694398,0.996770,"[L-Isoleucine, Hexanoate, Ornithine, D-Gluconi...",69,0.000,19
FBA3,0.000000,5.694398,0.996770,"[L-Isoleucine, Hexanoate, Ornithine, D-Gluconi...",69,0.000,19
PFK,0.000000,5.694398,4.697628,"[L-Isoleucine, Hexanoate, Ornithine, D-Gluconi...",60,0.000,3
FBA,0.000000,5.694398,4.697628,"[L-Isoleucine, Hexanoate, Ornithine, D-Gluconi...",59,0.000,2
SUCDi,2.834378,29.808212,2.834794,"[(S)-Propanediol, Glycolate, Glycerol3-phospha...",7,2.894,1
NADH16pp,0.000000,26.971191,26.970191,"[L-Isoleucine, Hexanoate, Ornithine, D-Gluconi...",63,8.001,1


Analyzing Product : L-Isoleucine


,lower,upper,pfba,products_within_bounds,number_within_bounds,mode,no_mode
FRD2,0.000000,14.282777,0.000000,"[Ethanolamine, Hexanoate, Ornithine, D-Gluconi...",69,0.000,68
NADH17pp,0.000000,14.281135,0.000000,"[Ethanolamine, Hexanoate, Ornithine, D-Gluconi...",69,0.000,58
TALA,-1.286078,2.696863,0.000000,"[Ethanolamine, Hexanoate, Ornithine, D-Gluconi...",64,0.000,48
PFK_3,0.000000,3.982941,2.696863,"[Ethanolamine, Hexanoate, Ornithine, D-Gluconi...",69,0.000,19
FBA3,0.000000,3.982941,2.696863,"[Ethanolamine, Hexanoate, Ornithine, D-Gluconi...",69,0.000,19
PFK,0.000000,3.982941,1.286078,"[Hexanoate, Ornithine, D-Gluconic Acid, L-Phen...",44,0.000,3
FBA,0.000000,3.982941,1.286078,"[Hexanoate, Ornithine, D-Gluconic Acid, L-Phen...",43,0.000,2
SUCDi,0.874878,15.158655,0.875293,"[Ethanolamine, D-Gluconic Acid, L-Phenylalanin...",43,0.938,1
NADH16pp,0.000000,14.281135,14.281135,"[Hexanoate, D-Gluconic Acid, (S)-Propanediol, ...",26,8.001,1


Analyzing Product : Dihydroxyacetone


...

,lower,upper,pfba,products_within_bounds,number_within_bounds,mode,no_mode
FRD2,0.000000,8.341565,0.000000,"[Ethanolamine, L-Isoleucine, Hexanoate, Ornith...",69,0.000,68
NADH17pp,0.000000,8.339521,0.000000,"[Ethanolamine, L-Isoleucine, Hexanoate, Ornith...",69,0.000,58
TALA,-1.867844,0.490248,0.000000,"[Ethanolamine, L-Isoleucine, Hexanoate, Ornith...",68,0.000,48
PFK_3,0.000000,2.358092,0.490248,"[Ethanolamine, Hexanoate, Ornithine, D-Gluconi...",65,0.000,19
FBA3,0.000000,2.358092,0.490248,"[Ethanolamine, Hexanoate, Ornithine, D-Gluconi...",65,0.000,19
PFK,0.000000,2.358092,1.866844,"[L-Isoleucine, Hexanoate, D-Gluconic Acid, 5-M...",21,0.000,3
FBA,0.000000,2.358092,1.866844,"[L-Isoleucine, Hexanoate, D-Gluconic Acid, 5-M...",20,0.000,2
SUCDi,0.737773,9.080338,0.737773,"[Ethanolamine, L-Isoleucine, D-Gluconic Acid, ...",42,0.860,1
NADH16pp,0.000000,8.339521,8.339521,[Ethanol],1,8.001,1


Analyzing Product : Succinate


,lower,upper,pfba,products_within_bounds,number_within_bounds,mode,no_mode
TALA,-6.279826,0.176663,0.000000,"[Ethanolamine, L-Isoleucine, Hexanoate, Ornith...",68,0.000,48
PFK_3,0.000000,6.456489,0.176663,"[Ethanolamine, L-Isoleucine, Hexanoate, Ornith...",69,0.000,19
FBA3,0.000000,6.456489,0.176663,"[Ethanolamine, L-Isoleucine, Hexanoate, Ornith...",69,0.000,19
SUCDi,0.000000,12.371933,0.000000,"[Ethanolamine, L-Isoleucine, Hexanoate, Ornith...",69,0.000,16
PFK,0.000000,6.456489,6.279826,"[Ethanolamine, L-Isoleucine, Hexanoate, Ornith...",63,0.000,3
FBA,0.000000,6.456489,6.279826,"[Ethanolamine, L-Isoleucine, Hexanoate, Ornith...",62,0.000,2
NADH16pp,0.000000,12.371933,12.370710,"[Hexanoate, D-Gluconic Acid, (S)-Propanediol, ...",17,8.001,1
NADH17pp,3.460310,15.833653,3.462943,[],0,NaN,0
FRD2,3.461943,15.835286,3.462943,[],0,NaN,0


Analyzing Product : Glycine


,lower,upper,pfba,products_within_bounds,number_within_bounds,mode,no_mode
FRD2,0.000000,14.096731,0.000000,"[Ethanolamine, L-Isoleucine, Hexanoate, Ornith...",69,0.000,68
NADH17pp,0.000000,14.093507,0.000000,"[Ethanolamine, L-Isoleucine, Hexanoate, Ornith...",69,0.000,58
PFK_3,0.000000,6.707945,0.000000,"[Ethanolamine, L-Isoleucine, Hexanoate, Ornith...",69,0.000,18
FBA3,0.000000,6.707945,0.000000,"[Ethanolamine, L-Isoleucine, Hexanoate, Ornith...",69,0.000,18
SUCDi,0.000000,14.097955,0.000000,"[Ethanolamine, L-Isoleucine, Hexanoate, Ornith...",69,0.000,16
PFK,0.000000,6.707945,6.707945,"[Ethanolamine, L-Isoleucine, Hexanoate, Ornith...",67,0.000,3
TALA,-6.774018,-0.066072,-0.066072,"[Glycolate, Acetate, L-Idonic Acid, γ-Aminobut...",12,-0.066,2
FBA,0.000000,6.707945,6.707945,"[Ethanolamine, L-Isoleucine, Hexanoate, Ornith...",66,0.000,2
NADH16pp,0.000000,14.093507,14.093507,"[Hexanoate, D-Gluconic Acid, (S)-Propanediol, ...",23,8.001,1


In [67]:
# Deepcopy parsed_fva and fva to perform product specific constrained fva
temp_fva = deepcopy(fva_fourth)
temp_parsed = deepcopy(parsed_fva_4)

In [68]:
deletable_reactions = {}
for product in metabolite_names_dict.values():
    temp = temp_dict[product][temp_dict[product]['mode']==0].index
    for reaction in temp:
        if (parsed_fva_4['flexible']['bounds'][product].loc[reaction]['lb']<=0) and (parsed_fva_4['flexible']['bounds'][product].loc[reaction]['ub']>=0):
            deletable_reactions[reaction] = deletable_reactions.get(reaction,[])+[product]
df_dict = {'products':[products for products in deletable_reactions.values()],
           'no':[len(parsed_fva_4['flexible']['pfba'].loc[reaction][parsed_fva_4['flexible']['pfba'].loc[reaction].round(decimals=3)==0]) for reaction in deletable_reactions.keys()]}
deletable_reactions = pd.DataFrame(data=df_dict,index=deletable_reactions.keys())
deletable_reactions = deletable_reactions.sort_values(by='no',ascending=False)

In [94]:
deletable_reactions

,products,no
FRD2,"[Spermidine, L-Tryptophan, Thymidine, L-Phenyl...",69
DURIPP,"[Adenosine, Adenine]",69
DURIK1,"[Adenosine, Adenine]",69
MALS,"[L-Idonic Acid, 5-Ketogluconate]",67
PYNP2r,"[Adenosine, Adenine]",67
FLDR2,[5-Ketogluconate],66
NTD2,"[Adenosine, Adenine]",66
PUNP2,"[Adenosine, Adenine]",65
ICL,"[L-Idonic Acid, 5-Ketogluconate]",65
NTD6,"[Adenosine, Adenine]",65


In [71]:
# FIFTH PASS FVA -  IMPLEMENT PRODUCT SPECIFIC DELETIONS OR ZERO CONSTRAINTS
# 1. For each flexible product:
# 2. Filter reactions that have '0' as their most frequently occurring flux value
# 3. For each such reaction:
# 4. if 0 is in the flux range for that product:
# 5. Perform FVA with new constraints for that specific product
# 6. Update fva, parsed_fva, perform the 
# 7. Continue
prod_specific_deletions = {}
for i,product in enumerate(metabolite_names_dict.values()):
    print('Evaluating product ', i+1, ' of 70')
    for reaction in deletable_reactions.index:
        if reaction in temp_parsed['flexible']['products'].index:
            if ((temp_parsed['flexible']['bounds'][product].loc[reaction]['lb']<=0) and (temp_parsed['flexible']['bounds'][product].loc[reaction]['ub']>=0) and temp_parsed['flexible']['range'][product].loc[reaction]>0):
                print('Removing reaction: ',reaction)
                prod_specific_deletions[product]=prod_specific_deletions.get(product,[]) + [reaction]
                temp = reduced_fva(flux_dict_prod, product, unused_reactions=temp_parsed['unused_reactions_set'].union(set(prod_specific_deletions.get(product))),
                                    used_reactions=temp_parsed['used_reactions_set'], template_model=model, slack=0.001)
                temp_fva[product]=temp[product]
                temp_fva,temp_parsed = global_fva_parser(temp_fva,flex_cutoff=0.008)



Evaluating product  1  of 70
Removing reaction:  FRD2
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpodpnsfos.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  TALA
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpjbq7g4z3.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating product  2  of 70
Removing reaction:  FRD2
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp91tzm14t.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  TALA
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp3y7ebsqg.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating product  3  of 70
Removing reaction:  FRD2
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpro6ycg9x.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  TALA
Read LP forma

Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  FBA3
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpt7x2tcji.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating product  22  of 70
Removing reaction:  FRD2
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpcb52fv_z.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  TALA
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpnuqinuc1.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating product  23  of 70
Removing reaction:  FRD2
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpxc96f_uz.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  TALA
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmph3hqa1z0.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonz

Removing reaction:  TALA
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp6scdzhtf.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating product  43  of 70
Removing reaction:  FRD2
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp9h9el04s.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  TALA
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmplrszgzbo.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating product  44  of 70
Removing reaction:  FRD2
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp2666dypl.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  TALA
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpp1hvbxwm.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating product  45  of 70
Removing reaction:  FRD2
Read LP fo

Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpxte9gxrz.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  FBA3
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp9ak4uvs3.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating product  65  of 70
Removing reaction:  FRD2
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpoctlx4ka.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  FBA3
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpuxpxdv5e.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating product  66  of 70
Removing reaction:  FRD2
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpn9w8v6uf.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  FBA3
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\

In [72]:
fva_fifth = deepcopy(temp_fva)

In [73]:
pickling_on = open("fva_fifth.pickle","wb")
pickle.dump(fva_fifth, pickling_on)
pickling_on.close() 

In [74]:
pickle_off = open("fva_fifth.pickle","rb")
fva_fifth = pickle.load(pickle_off)
pickle_off.close()

In [75]:
reactions_set = set([reaction.id for reaction in model.reactions])
relevant_reactions_set = set(relevant_reactions)
irrelevant_reactions_set = reactions_set-relevant_reactions_set

fva_fifth,parsed_fva_5 = global_fva_parser(fva_fifth,flex_cutoff=0.015)

print("Total Reactions: ",len(reactions_set), "\n")
print("Irrelevant_reactions: ",len(irrelevant_reactions_set))
print("Relevant Reactions: ",len(relevant_reactions_set), "\n")
print("Inflexible Reactions: ",len(parsed_fva_5['inflexible_reactions_set']))
print("Unused Inflexible Reactions: ",len(parsed_fva_5['inflexible_reactions_set'].intersection(parsed_fva_5['unused_reactions_set'])))
print("Used Inflexible Reactions: ",len(parsed_fva_5['inflexible_reactions_set'].intersection(parsed_fva_5['used_reactions_set'])), "\n")
print("Flexible Reactions: ",len(parsed_fva_5['flexible_reactions_set']))
print("Unused Flexible Reactions: ",len(parsed_fva_5['flexible_reactions_set'].intersection(parsed_fva_5['unused_reactions_set'])))
print("Used Flexible Reactions: ",len(parsed_fva_5['flexible_reactions_set'].intersection(parsed_fva_5['used_reactions_set'])))


Total Reactions:  2583 

Irrelevant_reactions:  1090
Relevant Reactions:  1493 

Inflexible Reactions:  1493
Unused Inflexible Reactions:  1120
Used Inflexible Reactions:  373 

Flexible Reactions:  0
Unused Flexible Reactions:  0
Used Flexible Reactions:  0


In [76]:
min_biomass = min(parsed_fva_5['inflexible']['pfba'].loc['BIOMASS_Ec_iJO1366_core_53p95M'])
for product in fva_fifth.keys():
    cutoff = 0.001*parsed_fva_5['inflexible']['pfba'][product].loc['BIOMASS_Ec_iJO1366_core_53p95M']/min_biomass
    fva_fifth[product]['pfba'][abs(fva_fifth[product]['pfba'])<=cutoff]=0
    fva_fifth[product]['lb'][abs(fva_fifth[product]['lb'])<=cutoff]=0
    fva_fifth[product]['ub'][abs(fva_fifth[product]['ub'])<=cutoff]=0
    fva_fifth[product]['range'] = fva_fifth[product]['ub'] - fva_fifth[product]['lb']
    fva_fifth[product]['range'][abs(fva_fifth[product]['range'])<=cutoff]=0

In [77]:
reactions_set = set([reaction.id for reaction in model.reactions])
relevant_reactions_set = set(relevant_reactions)
irrelevant_reactions_set = reactions_set-relevant_reactions_set

fva_final,parsed_fva_final = global_fva_parser(fva_fifth,flex_cutoff=0.015)

print("Total Reactions: ",len(reactions_set), "\n")
print("Irrelevant_reactions: ",len(irrelevant_reactions_set))
print("Relevant Reactions: ",len(relevant_reactions_set), "\n")
print("Inflexible Reactions: ",len(parsed_fva_final['inflexible_reactions_set']))
print("Unused Inflexible Reactions: ",len(parsed_fva_final['inflexible_reactions_set'].intersection(parsed_fva_final['unused_reactions_set'])))
print("Used Inflexible Reactions: ",len(parsed_fva_final['inflexible_reactions_set'].intersection(parsed_fva_final['used_reactions_set'])), "\n")
print("Flexible Reactions: ",len(parsed_fva_final['flexible_reactions_set']))
print("Unused Flexible Reactions: ",len(parsed_fva_final['flexible_reactions_set'].intersection(parsed_fva_final['unused_reactions_set'])))
print("Used Flexible Reactions: ",len(parsed_fva_final['flexible_reactions_set'].intersection(parsed_fva_final['used_reactions_set'])))


Total Reactions:  2583 

Irrelevant_reactions:  1090
Relevant Reactions:  1493 

Inflexible Reactions:  1493
Unused Inflexible Reactions:  1125
Used Inflexible Reactions:  368 

Flexible Reactions:  0
Unused Flexible Reactions:  0
Used Flexible Reactions:  0


In [87]:
pickling_on = open("fva_final_prod.pickle","wb")
pickle.dump(fva_final, pickling_on)
pickling_on.close() 

In [88]:
pickle_off = open("fva_final_prod.pickle","rb")
fva_final = pickle.load(pickle_off)
pickle_off.close()

In [89]:
pickling_on = open("parsed_fva_final_prod.pickle","wb")
pickle.dump(parsed_fva_final, pickling_on)
pickling_on.close() 

In [90]:
pickle_off = open("parsed_fva_final_prod.pickle","rb")
parsed_fva_final = pickle.load(pickle_off)
pickle_off.close()

In [82]:
parsed_fva_final['flexible']['products']

,flex,flex_no,inflex_used,inflex_used_no,inflex_unused,inflex_unused_no


In [83]:
len(parsed_fva_final['used_reactions_set'])

368

In [86]:
parsed_fva_final['inflexible']['pfba']

,Spermidine,L-Tryptophan,Thymidine,L-Phenylalanine,Adenosine,Indole,L-Tyrosine,Inosine,Xanthosine,Cytidine,Uridine,Quinate,L-Isoleucine,L-Lysine,Hexanoate,L-Leucine,L-Arginine,L-Histidine,D-Gluconic Acid,L-Idonic Acid,5-Ketogluconate,Citric Acid,Agmatine,Ornithine,L-Proline,L-Valine,Thymine,Adenine,Guanine,Hypoxanthine,O-Acetyl-L-serine,Xanthine,L-Glutamate,α-Ketoglutarate,Putrescine,L-Threonine,γ-Aminobutyrate,L-Homoserine,Allantoin,Succinate,Uracil,L-Asparagine,L-Malate,L-Aspartate,L-Cysteine,Glycerol3-phosphate,3-Hydroxypropanoate,(S)-Propanediol,(R)-Propanediol,D-Alanine,Glycerol,(R)-Glycerate,Glyceraldehyde,L-Lactate,Dihydroxyacetone,L-Alanine,D-Lactate,L-Serine,Pyruvate,Ethanolamine,Ethanol,Acetaldehyde,Glycine,Acetate,Glycolate,Urea,Formate,Glutathione,5-Methylthioribose,Cadaverine
ACOAD1f,-0.110219,-0.116049,-0.115321,-0.118200,-0.118623,-0.122229,-0.119994,-0.118066,-0.118176,-0.118101,-0.118781,-0.118938,-0.120362,-0.121685,-2.952742,-0.122742,-0.119322,-0.119745,-0.118968,-0.119906,-0.119622,-0.119168,-0.120474,-0.120228,-0.116874,-0.119437,-0.116437,-0.118953,-0.119848,-0.120806,-0.121791,-0.122239,-0.118784,-0.117774,-0.122936,-0.120148,-0.120244,-0.119329,-0.120641,-0.118892,-0.118880,-0.120229,-0.118035,-0.118884,-0.123126,-0.120416,-0.112515,-0.119594,-0.119377,-0.120154,-0.122516,-0.119928,-0.117956,-0.118078,-0.119856,-0.115382,-0.119083,-0.120949,-0.117826,-0.120693,-0.114746,-0.121622,-0.119153,-0.119991,-0.119735,-0.116469,-0.123336,-0.118535,-0.117833,-0.123386
UMPK,0.117082,0.123275,2.016370,0.125560,0.126009,0.129840,0.127466,0.125417,0.125535,2.391865,0.126177,0.126344,0.127857,0.129261,0.125953,0.130384,0.126752,0.127201,0.126375,0.127372,0.127071,0.126588,0.127976,0.127714,0.124151,0.126874,0.123687,0.126360,0.127311,0.128328,0.129375,0.129850,0.126180,0.125107,0.130591,0.127629,0.127731,0.126759,0.128153,0.126294,0.126282,0.127715,0.125384,0.126286,0.130793,0.127914,0.119521,0.127041,0.126810,2.719121,0.130145,0.127395,0.125300,0.125430,0.127319,0.122567,0.126498,0.128480,0.125163,0.128208,0.121891,0.129195,0.126573,0.127462,0.127190,0.123721,0.131016,0.125916,0.125170,0.131069
DDPA,0.118035,1.754998,0.123499,2.081353,0.127035,2.154736,2.172726,0.126439,0.126557,0.126476,0.127205,3.002140,0.128898,0.130314,0.126979,0.131446,0.127785,0.128238,0.127405,0.128410,0.128106,0.127619,0.129018,0.128755,0.125162,0.127908,0.124695,0.127389,0.128348,0.129374,0.130429,0.130908,0.127208,0.126126,0.131654,0.128669,0.128772,0.127792,0.129197,0.127323,0.127311,0.128755,0.126406,0.127315,0.131858,0.128956,0.120495,0.128076,0.127843,0.128675,0.131205,0.128433,0.126321,0.126452,0.128356,0.123565,0.127528,0.129527,0.126182,0.129252,0.122884,0.130247,0.127604,0.128501,0.128227,0.124729,0.132083,0.126942,0.126190,0.132137
IG3PS,0.029341,0.030893,0.030699,0.031466,0.031578,0.032538,0.031943,0.031430,0.031459,0.031439,0.031620,0.031662,0.032041,0.032393,0.031564,0.032675,0.031765,3.158971,0.031670,0.031920,0.031844,0.031723,0.032071,0.032006,0.031113,0.031795,0.030997,0.031666,0.031905,0.032160,0.032422,0.032541,0.031621,0.031352,0.032726,0.031984,0.032010,0.031766,0.032116,0.031650,0.031647,0.032006,0.031422,0.031648,0.032777,0.032056,0.029952,0.031837,0.031779,0.031986,0.032615,0.031926,0.031401,0.031433,0.031907,0.030716,0.031701,0.032198,0.031366,0.032129,0.030546,0.032377,0.031720,0.031943,0.031874,0.031005,0.032833,0.031555,0.031368,0.032846
FACOAE161,0.046589,0.049053,0.048745,0.049962,0.050141,0.051665,0.050721,0.049906,0.049952,0.049920,0.050208,0.050274,0.050876,0.051435,0.050119,0.051882,0.050437,0.050616,0.050287,0.050683,0.050564,0.050371,0.050924,0.050820,0.049402,0.050485,0.049217,0.050281,0.050659,0.051064,0.051480,0.051669,0.050209,0.049782,0.051964,0.050786,0.050826,0.050439,0.050994,0.050255,0.050250,0.050820,0.049892,0.050251,0.052045,0.050899,0.047559,0.050552,0.050460,0.050788,0.051787,0.050693,0.049859,0.049911,0.050662,0.048771,0.050336,0.051124,0.049804,0.051016,0.048502,0.051409,0.050365,0.050719,